In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [11]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'd-aea-model'
id_name = 'id'
target = 'ACTION'

file_name = "./d-aea/test_Amazon.com-Employee Access Challenge.csv"
test_file_name = "./d-aea/test_Amazon.com-Employee Access Challenge-no-label.csv"

kaggle_file_name = "./d-aea/kaggle-test-Amazon.com-Employee Access Challenge.csv"
kaggle_test_file_name = "./d-aea/kaggle-test-Amazon.com-Employee Access Challenge-no-label.csv"

kaggle_pred_file_name = "./d-aea/kaggle-test-Amazon.com-Employee Access Challenge-predictions.csv"

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

   RESOURCE  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  ROLE_DEPTNAME  ROLE_TITLE  \
0     75078   13876         117961         118386         118404      118685   
1     43085    1271         117961         118327         118507      118890   
2     78839     795         117961         118225         120551      117905   
3     38725   14674          11146         118491         117920      118568   
4     32270    5113         117961         118300         119890      120497   

   ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  
0            120316       308574     118687  
1            138290       118398     118892  
2            117906       290919     117908  
3            163031        19721     118570  
4            258776       118424     120499  
(3277, 9)


In [4]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)
    count = count + 1


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.9999958566545502, 0.9999752757503302, 0.9998424704826903, 0.9997956361476099, 0.9999907638894001, 0.9999926456512185, 0.999999965907783, 0.9999906389011812, 0.995082697365433, 0.9995763158367481]
[4.143345449847402e-06, 2.4724249669816345e-05, 0.00015752951730974019, 0.00020436385239008814, 9.23611059988616e-06, 7.354348781518638e-06, 3.409221704941956e-08, 9.361098818772007e-06, 0.004917302634567022, 0.00042368416325189173]


In [19]:
print(pred_label[:20])
print(pred_neg_prpb[:20])
print(pred_pos_prpb[:20])


import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print(np.mean(y_true==y_pred))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_neg_prpb, pos_label=1)
metrics.auc(fpr, tpr)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.9999958566545502, 0.9999752757503302, 0.9998424704826903, 0.9997956361476099, 0.9999907638894001, 0.9999926456512185, 0.999999965907783, 0.9999906389011812, 0.995082697365433, 0.9995763158367481, 0.002267122268676758, 0.9987954893149436, 0.9990924974554218, 0.9678932316601276, 0.9940431285649538, 0.9999994503787093, 0.9999910176347839, 0.9999984252493732, 0.9997183233208489, 0.9869777821004391]
[4.143345449847402e-06, 2.4724249669816345e-05, 0.00015752951730974019, 0.00020436385239008814, 9.23611059988616e-06, 7.354348781518638e-06, 3.409221704941956e-08, 9.361098818772007e-06, 0.004917302634567022, 0.00042368416325189173, 0.9977328777313232, 0.001204510685056448, 0.0009075025445781648, 0.03210676833987236, 0.005956871435046196, 5.496212907019071e-07, 8.98236521607032e-06, 1.5747506267871358e-06, 0.0002816766791511327, 0.013022217899560928]
0.9737564845895637


0.8993278933717034

In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

   RESOURCE  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  ROLE_DEPTNAME  ROLE_TITLE  \
0     78766   72734         118079         118080         117878      117879   
1     40644    4378         117961         118327         118507      118863   
2     75443    2395         117961         118300         119488      118172   
3     43219   19986         117961         118225         118403      120773   
4     42093   50015         117961         118343         119598      118422   

   ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  
0            118177        19721     117880  
1            122008       118398     118865  
2            301534       249618     118175  
3            136187       118960     120774  
4            300136       118424     118425  
(58921, 9)


In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [21]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

   id  ACTION
0   1       1
1   2       1
2   3       1
3   4       1
4   5       1
(58921, 2)


In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}